In [1]:
import os
import sys
home_dir = "../"
sys.path.append(home_dir)

import torch
import pandas as pd
import numpy as np
from Bio import SeqIO

In [2]:
os.listdir(home_dir+"data/SCOPe/downloads_at_ths/")

['astral-scopedom-seqres-gd-sel-gs-bib-10-2.08.fa',
 'astral-scopedom-seqres-gd-sel-gs-bib-100-2.08.fa',
 'astral-scopedom-seqres-gd-sel-gs-bib-20-2.08.fa',
 'astral-scopedom-seqres-gd-sel-gs-bib-25-2.08.fa',
 'astral-scopedom-seqres-gd-sel-gs-bib-30-2.08.fa',
 'astral-scopedom-seqres-gd-sel-gs-bib-35-2.08.fa',
 'astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.fa',
 'astral-scopedom-seqres-gd-sel-gs-bib-50-2.08.fa',
 'astral-scopedom-seqres-gd-sel-gs-bib-70-2.08.fa',
 'astral-scopedom-seqres-gd-sel-gs-bib-90-2.08.fa',
 'astral-scopedom-seqres-gd-sel-gs-bib-95-2.08.fa']

In [3]:
# from ESM1b exclude these: Rossman-like folds (c.2-c.5, c.27 and 28, c.30 and 31) and four- to eight-bladed β-propellers (b.66-b.70) 
to_exclude = ['c.2','c.3','c.4','c.5','c.27','c.28', 'c.30','c.31','b.66','b.67','b.68','b.69','b.70']

In [4]:
print(f"th\tn_datapoints\tn_unique_folds\tn_unique_superfamilies\tn_unique_families")
apply_filters = True
for fn in os.listdir(home_dir+"data/SCOPe_downloads/"):
    # fn = 'astral-scopedom-seqres-gd-sel-gs-bib-100-2.08.fa'
    data = []
    th = fn.split('-')[-2]
    i=0
    for record in SeqIO.parse(home_dir+"data/SCOPe_downloads/"+fn, "fasta"):
        # print(record)
        sid = str(record.id).strip()
        seq = str(record.seq).strip().upper()

        desc = record.description.split()

        sccs = desc[1]
        sccs_list = sccs.strip().split(".")
        cls_label, fold, superfamily, family = sccs_list[0], ".".join(sccs_list[:2]), ".".join(sccs_list[:3]), ".".join(sccs_list[:4])
        # cls_label, fold, superfamily, family = str(cls_label), str(fold), str(superfamily), str(family)

        span = desc[2]

        description = " ".join(desc[3:])

        # filters
        if apply_filters:
            if (sccs.count(".")!=3) or (len(span.split(','))!=1) or (".".join(sccs.split(".")[:2]) in to_exclude): 
                # if sccs label is not in this format: cls_label.fold.superfamily.family
                # if the data point comes from different spans
                # if folds are Rossman-like folds (c.2-c.5, c.27 and 28, c.30 and 31) and four- to eight-bladed β-propellers (b.66-b.70); from esm1b and Söding & Remmert, 2011)
                i+=1
                continue

        data_point = dict(sid=sid, span=span, sccs=sccs, cls_label=cls_label, fold=fold, superfamily=superfamily, family=family, seq=seq, description=description)
        data.append(data_point)

        # if i%1000==0: print(i)

        i+=1
        # break

    data_df = pd.DataFrame.from_dict(data)
    data_df['seq_len']=data_df['seq'].apply(len)

    data_df.to_csv(home_dir+"data/SCOPe/processed_at_th/th_"+str(th)+".tsv", sep="\t", index=False, header=True)

    
    total = data_df.shape[0]
    n_folds = data_df["fold"].unique().shape[0]
    n_superfamilies = data_df["superfamily"].unique().shape[0]
    n_families = data_df["family"].unique().shape[0]
    print(f"{th}\t{total}\t{n_folds}\t{n_superfamilies}\t{n_families}")
    # break

# print(data_df["cls_label"].unique(), data_df["cls_label"].unique().shape[0])

th	n_datapoints	n_unique_folds	n_unique_superfamilies	n_unique_families
10	6784	1220	1985	3999
100	89132	1231	1999	4863
20	7547	1229	1995	4116
25	8617	1229	1996	4197
30	10370	1229	1997	4310
35	12346	1229	1997	4432
40	14280	1230	1998	4521
50	17819	1230	1998	4695
70	23349	1230	1998	4791
90	30045	1230	1998	4817
95	33771	1230	1998	4826
